<a href="https://colab.research.google.com/github/Ashvinmax/tuner/blob/main/Welcome_to_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
# prompt: Require Dishes comparison from Planned and Actual datewise and Mealtime wise, where  I can  upload both the excel files and provide another compariossion.xlsx for download with results

import pandas as pd
from google.colab import files

def compare_dishes(planned_file, actual_file, output_file):
    """
    Compares dishes from planned and actual Excel files datewise and mealtime wise.

    Args:
        planned_file (str): Path to the planned Excel file.
        actual_file (str): Path to the actual Excel file.
        output_file (str): Path to save the comparison results Excel file.
    """
    try:
        # Read the Excel files
        df_planned = pd.read_excel(planned_file)
        df_actual = pd.read_excel(actual_file)

        # Ensure date and mealtime columns exist and are in the correct format
        for df, file_type in [(df_planned, 'planned'), (df_actual, 'actual')]:
            if 'Date' not in df.columns:
                print(f"Error: 'Date' column not found in {file_type} file.")
                return
            if 'Mealtime' not in df.columns:
                print(f"Error: 'Mealtime' column not found in {file_type} file.")
                return
            if 'Dish' not in df.columns:
                print(f"Error: 'Dish' column not found in {file_type} file.")
                return

            # Convert 'Date' column to datetime objects (handle potential errors)
            try:
                df['Date'] = pd.to_datetime(df['Date'])
            except Exception as e:
                print(f"Error converting 'Date' column in {file_type} file: {e}")
                print(f"Please ensure the 'Date' column is in a recognizable format.")
                return

        # Merge the dataframes on Date and Mealtime
        # Using a 'left' merge on planned allows us to see planned dishes that weren't in actual
        comparison_df = pd.merge(
            df_planned,
            df_actual,
            on=['Date', 'Mealtime', 'Dish'],
            how='left',
            indicator=True
        )

        # Determine the comparison result
        comparison_df['Comparison'] = comparison_df['_merge'].apply(
            lambda x: 'Match' if x == 'both' else 'Planned but not Actual'
        )

        # Find dishes in actual that were not in planned for the same Date and Mealtime
        actual_not_in_planned = pd.merge(
            df_actual,
            df_planned,
            on=['Date', 'Mealtime', 'Dish'],
            how='left',
            indicator=True
        )
        actual_not_in_planned = actual_not_in_planned[actual_not_in_planned['_merge'] == 'left_only']
        actual_not_in_planned['Comparison'] = 'Actual but not Planned'

        # Select relevant columns and rename for clarity
        comparison_df = comparison_df[['Date', 'Mealtime', 'Dish', 'Comparison']]
        actual_not_in_planned = actual_not_in_planned[['Date', 'Mealtime', 'Dish', 'Comparison']]

        # Combine the results
        final_comparison_df = pd.concat([comparison_df, actual_not_in_planned], ignore_index=True)

        # Sort for better readability
        final_comparison_df = final_comparison_df.sort_values(by=['Date', 'Mealtime', 'Dish']).reset_index(drop=True)

        # Save the comparison results to an Excel file
        final_comparison_df.to_excel(output_file, index=False)

        print(f"Comparison completed. Results saved to '{output_file}'")

        # Offer the file for download
        files.download(output_file)

    except FileNotFoundError as e:
        print(f"Error: File not found - {e}")
        print("Please make sure the file paths are correct.")
    except Exception as e:
        print(f"An error occurred: {e}")

# --- Main part to upload files and run the comparison ---

# Upload planned and actual files
print("Please upload the planned dishes Excel file:")
uploaded_planned = files.upload()
planned_filename = list(uploaded_planned.keys())[0]

print("\nPlease upload the actual dishes Excel file:")
uploaded_actual = files.upload()
actual_filename = list(uploaded_actual.keys())[0]

# Define the output filename
output_filename = 'comparison.xlsx'

# Run the comparison function
compare_dishes(planned_filename, actual_filename, output_filename)


Please upload the planned dishes Excel file:


Saving Planned.xlsx to Planned (21).xlsx

Please upload the actual dishes Excel file:


Saving Actual.xlsx to Actual (16).xlsx
Comparison completed. Results saved to 'comparison.xlsx'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<div class="markdown-google-sans">

## More resources

### Working with notebooks in Colab

</div>

- [Overview of Colab](/notebooks/basic_features_overview.ipynb)
- [Guide to markdown](/notebooks/markdown_guide.ipynb)
- [Importing libraries and installing dependencies](/notebooks/snippets/importing_libraries.ipynb)
- [Saving and loading notebooks in GitHub](https://colab.research.google.com/github/googlecolab/colabtools/blob/main/notebooks/colab-github-demo.ipynb)
- [Interactive forms](/notebooks/forms.ipynb)
- [Interactive widgets](/notebooks/widgets.ipynb)

<div class="markdown-google-sans">

<a name="working-with-data"></a>
### Working with data
</div>

- [Loading data: Drive, Sheets and Google Cloud Storage](/notebooks/io.ipynb)
- [Charts: visualising data](/notebooks/charts.ipynb)
- [Getting started with BigQuery](/notebooks/bigquery.ipynb)

<div class="markdown-google-sans">

### Machine learning

<div>

These are a few of the notebooks related to machine learning, including Google's online machine learning course. See the <a href="https://developers.google.com/machine-learning/crash-course/">full course website</a> for more.
- [Intro to Pandas DataFrame](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/pandas_dataframe_ultraquick_tutorial.ipynb)
- [Intro to RAPIDS cuDF to accelerate pandas](https://nvda.ws/rapids-cudf)
- [Getting started with cuML's accelerator mode](https://colab.research.google.com/github/rapidsai-community/showcase/blob/main/getting_started_tutorials/cuml_sklearn_colab_demo.ipynb)
- [Linear regression with tf.keras using synthetic data](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/linear_regression_with_synthetic_data.ipynb)

<div class="markdown-google-sans">

<a name="using-accelerated-hardware"></a>
### Using accelerated hardware
</div>

- [TensorFlow with GPUs](/notebooks/gpu.ipynb)
- [TPUs in Colab](/notebooks/tpu.ipynb)

<div class="markdown-google-sans">

<a name="machine-learning-examples"></a>

### Featured examples

</div>

- <a href="https://tensorflow.org/hub/tutorials/tf2_image_retraining">Retraining an Image Classifier</a>: Build a Keras model on top of a pre-trained image classifier to distinguish flowers.
- <a href="https://tensorflow.org/hub/tutorials/tf2_text_classification">Text Classification</a>: Classify IMDB film reviews as either <em>positive</em> or <em>negative</em>.
- <a href="https://tensorflow.org/hub/tutorials/tf2_arbitrary_image_stylization">Style Transfer</a>: Use deep learning to transfer style between images.
- <a href="https://tensorflow.org/hub/tutorials/retrieval_with_tf_hub_universal_encoder_qa">Multilingual Universal Sentence Encoder Q&amp;A</a>: Use a machine-learning model to answer questions from the SQuAD dataset.
- <a href="https://tensorflow.org/hub/tutorials/tweening_conv3d">Video Interpolation</a>: Predict what happened in a video between the first and the last frame.
